#Hidden Markov Models

####Jessica Morrise

In [1]:
%matplotlib inline
import numpy as np
import itertools
from matplotlib import pyplot as plt

The first toy HMM involves a simple weather model. We model 2 states, Hot and Cold, as well as 3 sizes of tree rings, Small, Medium, and Large.

In [402]:
A = np.array([[0.7,0.3],[0.4,0.6]])
B = np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
Pi = np.array([0,1])
N = 2
M = 3

dictA = {0:'H',1:'C'}
dictB = {0:'S',1:'M',2:'L'}
listToStrA = lambda l: ''.join([dictA[m] for m in l])
listToStrB = lambda l: ''.join([dictB[m] for m in l])

###Problem 1

We can use a naive method to calculate $P(\mathcal{O} | \lambda)$, by computing

$$P(\mathcal{O} | \lambda) = \sum_X \pi_{x_0}b_{x_0}(\mathcal{O}_0)a_{x_0,x_1}b_{x_1}(\mathcal{O}_1)...a_{x_{T-2},x_{T-1}}b_{x_{T-1}}(\mathcal{O}_{T-1})$$

By generating all possible observations and computing $P(O, X | \lambda)$ for each, we can calculate $P(\mathcal{O} | \lambda) = \sum_X P(\mathcal{O}, X | \lambda)$. This works, but is not the most efficient approach.

A better approach is to use the alpha pass, which is recursive:

$$\alpha_t(i) = \left[\sum_{j=0}^{N-1} \alpha_{t-1}(j)a_{ji}\right]$$

$$P(\mathcal{O} | \lambda) = \sum_{i=0}^{N-1} \alpha_{T-1}(i)$$

Functions for computing both are implemented below.

In [391]:
# Naive computation of P(O|lambda)
def pObsAndState(Obs,X):
    T = Obs.size
    p = Pi[X[0]]*B[X[0],Obs[0]]
    for t in xrange(1,T):
        p *= A[X[t-1],X[t]]*B[X[t],Obs[t]]
    return p
    
def naiveProb(Obs):
    T = Obs.size
    print "O = %s"%(listToStrB(Obs))
    #Generate all states
    totalProb = 0
    for state in itertools.product(range(N),repeat=T):
        prob = pObsAndState(Obs,state)
        totalProb += prob
        strState = listToStrA(state)
        print "%s: %.6f"%(strState,prob)
    print "p(Obs | A,B,Pi) = %.7f\n"%(totalProb)
    return totalProb

# Computation of P(O|lambda) using the recursive alpha pass method
def alphaPass(Obs):
    T = Obs.shape[0]
    print "O = %s"%(listToStrB(Obs))
    alpha = Pi*B[:,Obs[0]]
    for t in xrange(1,T):
        print "t=%d: "%(t-1) + str(alpha)
        alpha = A.T.dot(alpha)*B[:,Obs[t]]
    print "t=%d: "%(t) + str(alpha) + '\n'
    return alpha.sum()

The observations are $M$, $S$, $L$ ($T = 3$). Computing $P(\mathcal{O} | \lambda)$ both ways, we verify that both give the same output.

In [392]:
Obs = np.array([1,0,2])

print "Naive method:\n"
naive1 = naiveProb(Obs)

print "Alpha pass:\n"
alpha1 = alphaPass(Obs)
print "p(Obs) = ",alpha1

Naive method:

O = MSL
HHH: 0.000000
HHC: 0.000000
HCH: 0.000000
HCC: 0.000000
CHH: 0.002800
CHC: 0.000240
CCH: 0.016800
CCC: 0.005040
p(Obs | A,B,Pi) = 0.0248800

Alpha pass:

O = MSL
t=0: [ 0.   0.2]
t=1: [ 0.008  0.084]
t=2: [ 0.0196   0.00528]

p(Obs) =  0.02488


###Problem 2

In the Dynamic Programming sense, the most likely state sequence for $\mathcal{O} = M,S,L$ is the state sequence with the highest probability under the naive method. That is, $X = C,C,H$.

In the HMM sense, we look at the alpha pass at each time step. At $t=0$, the probability of $C$ is 0.2, while the probability of $H$ is 0, so we choose $x_0 = C$. Likewise, at $t=1$ and $t=2$ we choose $x_1 = C$ and $x_2 = H$. In this example, the most likely state sequence is the same for both DP and HMM.

###Problem 3

We use the same model, with $T = 4$. By the naive method, we verify that $\sum P(\mathcal{O}) = 1$. The helper functions below generate all possible combinations of observations.

In [393]:
#Helper functions
def naiveProbAll(T):
    total = 0
    #Not efficient (repeats calculations), but works.
    #Generate all possible observations
    for O in itertools.product(range(M),repeat=T):
        Obs = np.array(O)
        prob = naiveProb(Obs)
        total += prob
    return total

def alphaPassAll(T):
    total = 0
    #Generate all possible observations
    for O in itertools.product(range(M),repeat=T):
        Obs = np.array(O)
        alphaSum = alphaPass(Obs)
        total += alphaSum
    return total

In [394]:
totalProbability = pObsAll(4)
print "Naive method: Probability over all observations = %.5f"%(totalProbability)

O = SSSS
HHHH: 0.000000
HHHC: 0.000000
HHCH: 0.000000
HHCC: 0.000000
HCHH: 0.000000
HCHC: 0.000000
HCCH: 0.000000
HCCC: 0.000000
CHHH: 0.000137
CHHC: 0.000412
CHCH: 0.000235
CHCC: 0.002470
CCHH: 0.000823
CCHC: 0.002470
CCCH: 0.004939
CCCC: 0.051862
p(Obs | A,B,Pi) = 0.0633472

O = SSSM
HHHH: 0.000000
HHHC: 0.000000
HHCH: 0.000000
HHCC: 0.000000
HCHH: 0.000000
HCHC: 0.000000
HCCH: 0.000000
HCCC: 0.000000
CHHH: 0.000549
CHHC: 0.000118
CHCH: 0.000941
CHCC: 0.000706
CCHH: 0.003293
CCHC: 0.000706
CCCH: 0.019757
CCCC: 0.014818
p(Obs | A,B,Pi) = 0.0408856

O = SSSL
HHHH: 0.000000
HHHC: 0.000000
HHCH: 0.000000
HHCC: 0.000000
HCHH: 0.000000
HCHC: 0.000000
HCCH: 0.000000
HCCC: 0.000000
CHHH: 0.000686
CHHC: 0.000059
CHCH: 0.001176
CHCC: 0.000353
CCHH: 0.004116
CCHC: 0.000353
CCCH: 0.024696
CCCC: 0.007409
p(Obs | A,B,Pi) = 0.0388472

O = SSMS
HHHH: 0.000000
HHHC: 0.000000
HHCH: 0.000000
HHCC: 0.000000
HCHH: 0.000000
HCHC: 0.000000
HCCH: 0.000000
HCCC: 0.000000
CHHH: 0.000549
CHHC: 0.001646
CHCH: 0

But don't worry - the more efficient method, the alpha pass, can also be used to verify that $\sum P(\mathcal{O}) = 1$.

In [395]:
totalProbability2 = alphaPassAll(4)
print "Alpha pass: Probability over all observations = %.5f"%(totalProbability2)

O = SSSS
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.01372  0.12936]
t=3: [ 0.0061348  0.0572124]

O = SSSM
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.01372  0.12936]
t=3: [ 0.0245392  0.0163464]

O = SSSL
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.01372  0.12936]
t=3: [ 0.030674   0.0081732]

O = SSMS
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.05488  0.03696]
t=3: [ 0.00532   0.027048]

O = SSMM
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.05488  0.03696]
t=3: [ 0.02128   0.007728]

O = SSML
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.05488  0.03696]
t=3: [ 0.0266    0.003864]

O = SSLS
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.0686   0.01848]
t=3: [ 0.0055412  0.0221676]

O = SSLM
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.0686   0.01848]
t=3: [ 0.0221648  0.0063336]

O = SSLL
t=0: [ 0.   0.7]
t=1: [ 0.028  0.294]
t=2: [ 0.0686   0.01848]
t=3: [ 0.027706   0.0031668]

O = SMSS
t=0: [ 0.   0.7]
t=1: [ 0.112  0.084]
t=2: [ 0.0112  0.0588]
t=3: [ 0.003136  0

###Problem 4

The re-estimation formulae:

$$\gamma_t(i) = \frac{\alpha_t(i)a_{[i,:]} \cdot (b(\mathcal{O}_{t+1})\circ \beta_{t+1})}{P(\mathcal{O}| \lambda)}$$

$$\pi_i = \frac{\alpha_0(i)\beta_0(i)}{P(\mathcal{O}| \lambda)}$$

$$a_{ij} = \sum_{t=0}^{T-2} \frac{\alpha_t(i)a_{ij}b_j(\mathcal(O)_{t+1})\beta_{t+1}(j)}{P(O, X | \lambda)} \bigg/ \sum_{t=0}^{T-2} \frac{\alpha_t(i)a_{[i,:]} \cdot (b(\mathcal{O}_{t+1})\circ \beta_{t+1})}{P(\mathcal{O}| \lambda)}$$

$$b_j(k) = \sum_{\mathcal{O}_t=k} \frac{\alpha_t(i)a_{[i,:]} \cdot (b(\mathcal{O}_{t+1})\circ \beta_{t+1})}{P(\mathcal{O}| \lambda)} \bigg/ \sum_{t=0}^{T-1} \frac{\alpha_t(i)a_{[i,:]} \cdot (b(\mathcal{O}_{t+1})\circ \beta_{t+1})}{P(\mathcal{O}| \lambda)}$$

<hr/>

### HMM Solver

To keep everything together, we implement an HMM class to solve all three problems: alpha pass, beta pass, and re-estimation formulae. The class is initialized with $A$, $B$, and $\pi$, or else it is initialized with sizes and the parameters are chosen randomly. Operations are vectorized for better speed.

In [401]:
class vectorizedHMM(object):
    def __init__(self,A=None,B=None,pi=None,N=None,M=None):
        if A is None and N is None:
            raise ValueError("Need to pass in either A or N")
        elif A is None:
            #Initialize A and B to NxN and NxM, with random error.
            self.N = N
            self.M = M
            self.A = np.ones((N,N)) + np.random.normal(scale=0.4,size=(N,N))
            self.A /= np.vstack(self.A.sum(axis=1)) #Normalize each row
            self.B = np.ones((N,M)) + np.random.normal(scale=0.2,size=(N,M))
            self.B /= np.vstack(self.B.sum(axis=1)) #Normalize each row
            self.pi = np.ones(N)*(1./N) + np.random.normal(scale=0.2,size=N)
            self.pi /= self.pi.sum() #Normalize each row
        else:
            #A specific inital guess was provided
            self.A = A
            self.B = B
            self.pi = pi
            self.N = self.A.shape[0]
            self.M = self.B.shape[1]
            
        self.c = np.array([-np.inf])
        
    def alphaPass(self,Obs):
        #Vectorized version
        #The alpha pass algorithm
        T = Obs.shape[0]
        self.c = np.zeros(T) #Scaling factors
        alphaMatrix = np.zeros((T,self.N))
        for t in xrange(T):
            if t == 0:
                alpha_tilde = self.pi*self.B[:,Obs[0]]
            else:
                alpha_tilde = self.A.T.dot(alpha_hat)*self.B[:,Obs[t]]
            self.c[t] = 1./alpha_tilde.sum() #Calculate scaling factors
            alpha_hat = self.c[t]*alpha_tilde #scale alpha hat
            alphaMatrix[t,:] = alpha_hat
        return alphaMatrix
        
    def betaPass(self,Obs):
        #Vectorized version
        #The beta pass algorithm
        T = Obs.shape[0]
        betaMatrix = np.zeros((T,self.N))
        beta_hat = np.ones(self.N)*self.c[-1]
        betaMatrix[-1,:] = beta_hat
        for t in xrange(T-2,-1,-1):
            betaMatrix[t,:] = self.c[t]*self.A.dot(betaMatrix[t+1,:]*self.B[:,Obs[t+1]])
        return betaMatrix
        
    def reestimate(self,Obs):
        #Vectorized version
        #Use the alpha and beta pass to re-estimate A, B, and pi
        alphaMat = self.alphaPass(Obs)
        betaMat = self.betaPass(Obs)
        #Calculate gammas
        gammaMat = alphaMat*betaMat
        #Normalize the gammas
        gammaMat /= np.vstack(gammaMat.sum(axis=1))
        
        #Calculate the digammas
        #Vectorized version
        T = Obs.shape[0]
        D = np.array([self.A for A in xrange(T-1)])
        D *= np.expand_dims(alphaMat[:-1,:], axis=2)
        B = self.B[:,Obs[1:]].T*betaMat[1:,:]
        D *= np.expand_dims(B, axis=1)
        
        digammaSum = D.sum(axis=0)
         
        
        #Re-estimate A,B,pi using the reestimation formulae
        A_new =  digammaSum/np.vstack(gammaMat[:-1,:].sum(axis=0))
        B_new = np.zeros_like(self.B)
        for k in xrange(self.M):
            B_new[:,k] = gammaMat[Obs==k,:].sum(axis=0)/gammaMat.sum(axis=0) 
        pi_new = gammaMat[0,:]
        self.probableStates = np.argmax(gammaMat,axis=1)
        return A_new, B_new, pi_new
    
    def calcLogProb(self):
        #Calculate the log of the probability of Obs
        return -np.sum(np.log(self.c))
        
    def train(self,Obs,maxIters=100,thresh=1e-2):
        logProb = -np.inf
        logProbDiff = np.inf
        iters = 0
        while iters < maxIters and abs(logProbDiff) > thresh:
            self.A, self.B, self.pi = self.reestimate(Obs)
            logProbDiff =  self.calcLogProb() - logProb
            logProb = self.calcLogProb()
            print iters, logProb, logProbDiff
            print "States:",np.argmax(self.B,axis=0)
            iters += 1
        print "Completed after %d iterations"%(iters)
        
    def mostLikelyStateDiv(self):
        #For each possible observation, assign a most likely state based on B
        states = np.argmax(self.B,axis=0)
        letters = np.array([chr(x) for x in xrange(97,123)] + [' '])
        for i in xrange(self.N):
            print letters[states==i],'\n'

### Problem 9

We will now use our HMM solver to analyze hidden states and transition probabilities between letters in English. The observations come from the Brown corpus, a large sample of English text. First we load in the Brown corpus and take approximately the first 15,000 words (assuming, as per Wikipedia, that a word is 5.1 letters long on average). 15,000 is chosen because 50,000 was prohibitively time-consuming.

We map each character in the first 15,000 words to a number. The 26 letters plus space make a total of 27 possible observations. 

In [344]:
def wordsToNum(s):
    s = s.replace(' ','{') #ord('{') = ord('z')+1
    return np.array([ord(char)-97 for char in s]) #ord('a') = 97

with open('brown.txt','r') as f:
    wordsString =  f.readline()
    nWords = 15000
    wordLen = 5.1 #average length of English word
    wordsToTrain = wordsString[:int(nWords*(wordLen+1))]
    
wordsObs = wordsToNum(wordsToTrain) #Our observation sequence

Next, we initialize an HMM with 2 possible states and 27 possible observations. The observations are the characters, but the nature of the 2 states is unknown. The HMM runs for 100 observations. 
The printed output includes:
* Iteration number
* Log probability
* Change in log probability
* Most likely state for each observation - this is helpful in seeing how the model's probabilities develop over time

In [347]:
englishHMM = vectorizedHMM(N=2,M=27)
englishHMM.train(wordsObs,maxIters=100)

0 -304969.754386 inf
States: [1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 1 1 1 1 1 1 0 0 1]
1 -260302.847915 44666.9064717
States: [1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1]
2 -260270.461115 32.3868002052
States: [1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 1]
3 -260238.376409 32.0847051406
States: [1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 0]
4 -260204.964707 33.4117025871
States: [1 0 1 1 0 0 1 0 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0]
5 -260168.744213 36.220494036
States: [1 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0]
6 -260128.221819 40.522393847
States: [1 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0]
7 -260081.714639 46.5071798707
States: [1 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0]
8 -260027.095884 54.618755063
States: [1 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0]
9 -259961.388832 65.7070519013
States: [1 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0]
10 -259880.101646 81.2871858875
States: [1 0 1 1 0 0 1 0 1 0 1 

The model divides the observations into 2 states. It turns out that one state represents vowels, while the other represents consonants. We print out the characters grouped by most likely state.

In [397]:
letters = np.array([chr(x) for x in xrange(97,123)] + [' '])

def characterDiv(B):
    N = B.shape[0]
    states = np.argmax(B,axis=0)
    for i in xrange(N):
        print 'State {}: {}\n'.format(i+1,letters[states==i])
print "2 state HMM:\n"        
characterDiv(englishHMM.B)

2 state HMM:

State 1: ['a' 'e' 'i' 'o' 'u' ' ']

State 2: ['b' 'c' 'd' 'f' 'g' 'h' 'j' 'k' 'l' 'm' 'n' 'p' 'q' 'r' 's' 't' 'v' 'w'
 'x' 'y' 'z']



For comparison with the textbook, the final parameters of the HMM are displayed below.

In [398]:
print "Parameters for the 2 state HMM\n"

print 'Pi:\n', englishHMM.pi, '\n'

print 'A:\n', englishHMM.A, '\n'

print 'B: \n'
for i in xrange(27):
    print '%s   %.6f %.6f'%(letters[i], englishHMM.B[0,i], englishHMM.B[1,i])

Parameters for the 2 state HMM

Pi:
[  6.23312384e-162   1.00000000e+000] 

A:
[[ 0.26900993  0.73099007]
 [ 0.7168983   0.2831017 ]] 

B: 

a   0.136553 0.000575
b   0.000000 0.023444
c   0.000000 0.056412
d   0.000000 0.065893
e   0.213431 0.000534
f   0.000000 0.035847
g   0.000124 0.028691
h   0.008396 0.070063
i   0.124801 0.000000
j   0.000000 0.003464
k   0.001484 0.007116
l   0.000575 0.069983
m   0.000000 0.040783
n   0.000000 0.116439
o   0.129811 0.000000
p   0.000218 0.037171
q   0.000000 0.001429
r   0.000000 0.101221
s   0.000000 0.110789
t   0.001984 0.157614
u   0.043395 0.000000
v   0.000000 0.016906
w   0.000000 0.025284
x   0.000000 0.004070
y   0.000000 0.025175
z   0.000000 0.001082
    0.339228 0.000012


The meaning of the states is less clear when there are more than 2 states. They also tend to vary based on the initial guess.

Here are the results for a 3-state HMM:

In [377]:
englishHMM3 = vectorizedHMM(N=3,M=27)
englishHMM3.train(wordsObs,maxIters=100)

0 -297618.262612 inf
States: [0 1 1 0 2 0 0 0 0 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
1 -260272.353799 37345.9088124
States: [0 1 1 0 2 0 0 0 0 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
2 -260257.714709 14.6390901183
States: [0 1 1 0 2 0 0 0 0 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
3 -260237.704277 20.0104325071
States: [0 1 1 0 2 0 0 0 0 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
4 -260209.553467 28.1508097026
States: [2 1 1 0 2 0 0 0 0 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
5 -260168.962982 40.5904851658
States: [2 1 1 0 2 0 0 0 2 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
6 -260109.151303 59.8116790675
States: [2 1 1 0 2 0 0 0 2 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
7 -260019.253316 89.8979866665
States: [2 1 1 0 2 0 0 0 2 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
8 -259881.610484 137.642832004
States: [2 1 1 0 2 0 0 0 2 0 0 1 1 1 2 0 0 0 2 0 0 0 0 1 2 0 1]
9 -259667.268886 214.341598447
States: [2 0 1 0 2 0 0 0 2 0 0 1 1 1 2 0 0 0 2 0 2 0 0 1 2 0 1]
10 -259329.064171 338.204714758
States: [2 0 1 0 2 0 0 0 2 0

In [399]:
print "3 state HMM:\n"
characterDiv(englishHMM3.B)

3 state HMM:

State 1: ['d' 'g' 'h' 'j' 'k' 'l' 'm' 'q' 'v' 'y' 'z']

State 2: ['b' 'c' 'f' 'n' 'p' 'r' 's' 't' 'w' 'x']

State 3: ['a' 'e' 'i' 'o' 'u' ' ']



And the results for 4 states:

In [354]:
englishHMM4 = vectorizedHMM(N=4,M=27)
englishHMM4.train(wordsObs,maxIters=100)

0 -302681.399022 inf
States: [2 0 1 2 1 3 2 3 2 3 1 2 2 0 0 0 2 0 2 0 1 1 3 1 2 2 3]
1 -260286.242044 42395.1569776
States: [3 0 1 2 1 1 2 3 2 2 1 2 2 0 0 0 2 0 2 0 1 1 3 1 2 2 3]
2 -260273.570172 12.6718725199
States: [3 0 1 2 1 1 2 3 3 2 1 2 2 0 0 0 2 0 2 0 1 1 0 1 2 2 3]
3 -260256.927208 16.6429636709
States: [3 0 1 2 1 1 2 3 3 0 1 2 2 0 0 0 2 0 2 0 1 1 0 1 2 2 3]
4 -260234.460623 22.4665846369
States: [3 0 1 2 1 1 2 3 3 0 1 2 2 0 0 0 2 0 2 2 1 1 0 1 2 2 3]
5 -260203.376825 31.0837984327
States: [3 0 1 2 1 1 2 3 3 0 1 2 2 0 0 0 2 0 2 2 1 1 0 1 2 2 3]
6 -260159.340983 44.035841823
States: [3 0 1 2 1 1 2 3 3 0 1 2 2 0 0 0 2 0 2 2 3 0 0 1 2 2 3]
7 -260095.428029 63.9129536494
States: [3 0 2 2 1 1 2 3 3 0 1 2 2 0 0 0 2 0 2 2 3 0 0 1 2 2 3]
8 -260000.256681 95.1713484846
States: [3 0 2 2 1 1 2 3 3 0 1 2 2 0 0 0 2 0 2 2 3 0 0 1 2 2 3]
9 -259854.662282 145.594399143
States: [3 0 2 2 1 1 2 3 3 0 1 2 2 0 0 0 2 0 2 2 3 0 0 1 2 2 3]
10 -259625.939663 228.722618389
States: [3 0 2 2 1 2 2 3 3 0 

In [376]:
print "4 state HMM:\n"
characterDiv(englishHMM4.B)

4 state HMM:

State 1: ['b' 'c' 'n' 's' 't']

State 2: ['g' 'h' 'k' 'y']

State 3: ['d' 'f' 'j' 'l' 'm' 'p' 'q' 'r' 'v' 'w' 'x' 'z']

State 4: ['a' 'e' 'i' 'o' 'u' ' ']



###Problem 10

And now... we will attempt to use an HMM to break a cipher!

To speed up the process, we first train the HMM on the beginning of the ciphertext ($T = 10000$) for 50 iterations, hoping that the smaller amount of data will at least get the model parameters into the right ballpark. We then train for another 30 or so iterations on the entire ciphertext ($T = 69819$) to refine the model.

In [362]:
with open('ciphertext.txt','r') as f:
    cipher =  f.readline()
    
cipherObs = wordsToNum(cipher) #Observation sequence

In [366]:
#Train on part of the observation sequence to start
cipherHMM = vectorizedHMM(N=2, M=27)
cipherHMM.train(cipherObs[:10000],maxIters=50)

0 -32936.8998458 inf
States: [1 0 1 1 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
1 -28079.4362906 4857.46355522
States: [1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
2 -28076.1786762 3.25761440078
States: [1 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
3 -28072.1919693 3.98670689412
States: [1 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
4 -28067.211049 4.98092025258
States: [1 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
5 -28060.8870609 6.32398809995
States: [1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
6 -28052.7597887 8.12727223637
States: [1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1]
7 -28042.2278505 10.5319381476
States: [1 1 1 1 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 1]
8 -28028.5250486 13.7028018996
States: [1 1 1 1 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 1]
9 -28010.7242323 17.8008163438
States: [1 1 1 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 1 1 1 0 1]
10 -27987.8153714 22.9088609302
States: [1 1 1 1 1 0 0 1 1 0 

In [368]:
#Train on the entire sequence to refine
cipherHMM.train(cipherObs,maxIters=30)

0 -191982.643193 inf
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
1 -191975.917596 6.72559648706
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
2 -191970.07508 5.84251627943
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
3 -191965.02467 5.05040952159
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
4 -191960.674304 4.35036686927
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
5 -191956.935014 3.73929006426
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
6 -191953.723807 3.21120689253
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
7 -191950.965358 2.75844849236
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
8 -191948.592762 2.3725962262
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
9 -191946.547587 2.0451753391
States: [1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1]
10 -191944.779469 1.76811800839
States: [1 1 0 1 1 1 1 0 1 1 0 1

The results: Presumably, *a, e, i, o, u* and space are coded by *c, h, k, o, u*, and *v*.

In [370]:
characterDiv(cipherHMM.B)

State 1: ['c' 'h' 'k' 'o' 'u' 'v']

State 2: ['a' 'b' 'd' 'e' 'f' 'g' 'i' 'j' 'l' 'm' 'n' 'p' 'q' 'r' 's' 't' 'w' 'x'
 'y' 'z' ' ']



And the resulting parameters:

In [400]:
print 'Pi:\n', cipherHMM.pi, '\n'

print 'A:\n', cipherHMM.A, '\n'

print 'B: \n'
for i in xrange(27):
    print '%s   %.6f %.6f'%(letters[i], cipherHMM.B[0,i], cipherHMM.B[1,i])

Pi:
[  1.32280227e-216   1.00000000e+000] 

A:
[[ 0.28632373  0.71367627]
 [ 0.70722431  0.29277569]] 

B: 

a   0.000000 0.028801
b   0.000000 0.093589
c   0.201089 0.000000
d   0.000000 0.041120
e   0.000000 0.094416
f   0.000000 0.015826
g   0.000000 0.000599
h   0.041207 0.000343
i   0.000000 0.040550
j   0.000000 0.037128
k   0.374662 0.000006
l   0.000000 0.111583
m   0.000000 0.001226
n   0.001595 0.012877
o   0.107163 0.000000
p   0.006987 0.067902
q   0.000012 0.023257
r   0.000006 0.082576
s   0.002905 0.092108
t   0.000000 0.001226
u   0.131163 0.000000
v   0.132912 0.000006
w   0.000000 0.001597
x   0.000253 0.142044
y   0.000000 0.036215
z   0.000047 0.037366
    0.000000 0.037641


###Appendix A: What is this cipher anyway?

We can make a few guesses by comparing $B$ with our previous English language example. Probably the letter *k* corresponds to a space, since *k* has by far the highest probability here and space has by far the highest probability in the true model. Similarly, it would make sense to guess that *c* corresponds to *e*, and *h* corresponds to *u*. 

Based on what we see in problem 9, we might guess that *x* and *s*, which are the two consonants with the highest probabilities on the vowel side, correspond to *t* and *h*. We might also reasonably suppose that *v*, the 3rd most probable vowel here, corresponds with *a*, the 3rd most probable in the true model. Beyond that, things get really hazy. 

Below we replace ciphertext spaces with underscores, and then replace the cipher letters with the actual letters in uppercase. Let's see what comes out:

In [427]:
cipherGuess = cipher.replace(' ','_').replace('k',' ').replace('c','E').replace('h','U').replace('s','H')\
.replace('x','T').replace('v','A')
print cipherGuess[:1000]

THoe iuun oe pAbzEpj aulaEblEr _oTH HuiioTe Alr dbuy oTe qAzEe A bEArEb yAj roeaufEb yUaH ud THEob aHAbAaTEb Alr A poTTpE ud THEob HoeTubj dUbTHEb oldubyAToul _opp Apeu iE duUlr ol THE eEpEaToul dbuy THE bEr iuun ud _EeTyAbaH THAT HAe ApbEArj iEEl qUipoeHEr UlrEb THE ToTpE ud THE HuiioT THAT eTubj _Ae rEbofEr dbuy THE EAbpoEb aHAqTEbe ud THE bEr iuun auyqueEr ij iopiu HoyeEpd THE dobeT HuiioT Tu iEauyE dAyuUe ol THE _ubpr AT pAbzE Alr aAppEr ij Hoy THEbE Alr iAan AzAol eolaE THEj Tupr ud Hoe tuUblEj olTu THE EAeT Alr Hoe bETUbl Al ArfElTUbE _HoaH pATEb olfupfEr App THE HuiioTe ol THE zbEAT EfElTe ud THAT AzE THAT AbE HEbE bEpATEryAlj Hu_EfEb yAj _oeH Tu nlu_ yubE AiuUT THoe bEyAbnAipE qEuqpE dbuy THE uUTeET _HopE euyE yAj luT queeEee THE EAbpoEb iuun dub eUaH bEArEbe A dE_ luTEe ul THE yubE oyqubTAlT quolTe AbE HEbE auppEaTEr dbuy HuiioTpubE Alr THE dobeT ArfElTUbE oe iboEdpj bEaAppErHuiioTe AbE Al UluiTbUeofE iUT fEbj AlaoElT qEuqpE yubE lUyEbuUe dubyEbpj THAl THEj AbE TurAj dub THEj 

It looks like the words are split up into reasonable lengths, so we got the space right. We know *u* goes with some vowel, and the second word is 'iuun' - the only vowel left that it makes sense to double is *o*, so *u* $\rightarrow$ *o*. The 'oe' combination in the first few words hints that maybe *o* goes with *i* and *e* goes with *s*. Let's try it...

In [428]:
cipherGuess2 = cipherGuess.replace('u','O').replace('o','I').replace('e','S')
print cipherGuess2[:1000]

THIS iOOn IS pAbzEpj aOlaEblEr _ITH HOiiITS Alr dbOy ITS qAzES A bEArEb yAj rISaOfEb yUaH Od THEIb aHAbAaTEb Alr A pITTpE Od THEIb HISTObj dUbTHEb IldObyATIOl _Ipp ApSO iE dOUlr Il THE SEpEaTIOl dbOy THE bEr iOOn Od _ESTyAbaH THAT HAS ApbEArj iEEl qUipISHEr UlrEb THE TITpE Od THE HOiiIT THAT STObj _AS rEbIfEr dbOy THE EAbpIEb aHAqTEbS Od THE bEr iOOn aOyqOSEr ij iIpiO HIySEpd THE dIbST HOiiIT TO iEaOyE dAyOUS Il THE _Obpr AT pAbzE Alr aAppEr ij HIy THEbE Alr iAan AzAIl SIlaE THEj TOpr Od HIS tOUblEj IlTO THE EAST Alr HIS bETUbl Al ArfElTUbE _HIaH pATEb IlfOpfEr App THE HOiiITS Il THE zbEAT EfElTS Od THAT AzE THAT AbE HEbE bEpATEryAlj HO_EfEb yAj _ISH TO nlO_ yObE AiOUT THIS bEyAbnAipE qEOqpE dbOy THE OUTSET _HIpE SOyE yAj lOT qOSSESS THE EAbpIEb iOOn dOb SUaH bEArEbS A dE_ lOTES Ol THE yObE IyqObTAlT qOIlTS AbE HEbE aOppEaTEr dbOy HOiiITpObE Alr THE dIbST ArfElTUbE IS ibIEdpj bEaAppErHOiiITS AbE Al UlOiTbUSIfE iUT fEbj AlaIElT qEOqpE yObE lUyEbOUS dObyEbpj THAl THEj AbE TOrAj dOb THEj 

Now '\_ith' really makes it look like \_ corresponds to *w*, and 'Alr' all over the place suggest that *l* $\rightarrow$ *n*, *r* $\rightarrow$ *d*. That word with a *p* in the middle is probably 'title', so *p* goes with *l*, and is the second word supposed to be 'book'?

In [429]:
cipherGuess3 = cipherGuess2.replace('_','W').replace('l','N').replace('r','D').replace('p','L')\
.replace('i','B').replace('n','K')
print cipherGuess3[:1000]

THIS BOOK IS LAbzELj aONaEbNED WITH HOBBITS AND dbOy ITS qAzES A bEADEb yAj DISaOfEb yUaH Od THEIb aHAbAaTEb AND A LITTLE Od THEIb HISTObj dUbTHEb INdObyATION WILL ALSO BE dOUND IN THE SELEaTION dbOy THE bED BOOK Od WESTyAbaH THAT HAS ALbEADj BEEN qUBLISHED UNDEb THE TITLE Od THE HOBBIT THAT STObj WAS DEbIfED dbOy THE EAbLIEb aHAqTEbS Od THE bED BOOK aOyqOSED Bj BILBO HIySELd THE dIbST HOBBIT TO BEaOyE dAyOUS IN THE WObLD AT LAbzE AND aALLED Bj HIy THEbE AND BAaK AzAIN SINaE THEj TOLD Od HIS tOUbNEj INTO THE EAST AND HIS bETUbN AN ADfENTUbE WHIaH LATEb INfOLfED ALL THE HOBBITS IN THE zbEAT EfENTS Od THAT AzE THAT AbE HEbE bELATEDyANj HOWEfEb yAj WISH TO KNOW yObE ABOUT THIS bEyAbKABLE qEOqLE dbOy THE OUTSET WHILE SOyE yAj NOT qOSSESS THE EAbLIEb BOOK dOb SUaH bEADEbS A dEW NOTES ON THE yObE IyqObTANT qOINTS AbE HEbE aOLLEaTED dbOy HOBBITLObE AND THE dIbST ADfENTUbE IS BbIEdLj bEaALLEDHOBBITS AbE AN UNOBTbUSIfE BUT fEbj ANaIENT qEOqLE yObE NUyEbOUS dObyEbLj THAN THEj AbE TODAj dOb THEj 

And the appearance of 'hobbits' pretty much guarantees that all our guesses have been correct. Here's all of it:

In [437]:
cipherGuessFinal = cipherGuess3.replace('d','F').replace('b','R').replace('f','V').replace('j','Y').replace('z','G')\
.replace('y','M').replace('a','C').replace('t','J').replace('q','P').replace('m','Q').replace('f','Z').replace('w','X')
print cipherGuessFinal[:1000],'...'

THIS BOOK IS LARGELY CONCERNED WITH HOBBITS AND FROM ITS PAGES A READER MAY DISCOVER MUCH OF THEIR CHARACTER AND A LITTLE OF THEIR HISTORY FURTHER INFORMATION WILL ALSO BE FOUND IN THE SELECTION FROM THE RED BOOK OF WESTMARCH THAT HAS ALREADY BEEN PUBLISHED UNDER THE TITLE OF THE HOBBIT THAT STORY WAS DERIVED FROM THE EARLIER CHAPTERS OF THE RED BOOK COMPOSED BY BILBO HIMSELF THE FIRST HOBBIT TO BECOME FAMOUS IN THE WORLD AT LARGE AND CALLED BY HIM THERE AND BACK AGAIN SINCE THEY TOLD OF HIS JOURNEY INTO THE EAST AND HIS RETURN AN ADVENTURE WHICH LATER INVOLVED ALL THE HOBBITS IN THE GREAT EVENTS OF THAT AGE THAT ARE HERE RELATEDMANY HOWEVER MAY WISH TO KNOW MORE ABOUT THIS REMARKABLE PEOPLE FROM THE OUTSET WHILE SOME MAY NOT POSSESS THE EARLIER BOOK FOR SUCH READERS A FEW NOTES ON THE MORE IMPORTANT POINTS ARE HERE COLLECTED FROM HOBBITLORE AND THE FIRST ADVENTURE IS BRIEFLY RECALLEDHOBBITS ARE AN UNOBTRUSIVE BUT VERY ANCIENT PEOPLE MORE NUMEROUS FORMERLY THAN THEY ARE TODAY FOR THEY 

Sangat hebat.